In [3]:
import pandas as pd
from pandas import Series,DataFrame

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
titanic_df = pd.read_csv('train.csv')

In [5]:
def male_female_child(passenger):
    # 年齢と性別のデータを取得します。
    age,sex = passenger
    # 年齢を調べて16歳未満なら、子供。それ以外は性別をそのまま返します。
    if age < 16:
        return 3
    else:
        if sex == 'male':
            return 1
        else:
            return 2
    
# personという新しい列を追加します。
titanic_df['Person'] = titanic_df[['Age','Sex']].apply(male_female_child,axis=1)

In [6]:
titanic_df['Alone'] = titanic_df.Parch + titanic_df.SibSp

In [7]:
titanic_df['Alone'].loc[titanic_df['Alone'] >0] = 1
titanic_df['Alone'].loc[titanic_df['Alone'] == 0] = 0

/usr/local/pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
train_data = titanic_df.drop(['PassengerId','Name','Sex','SibSp','Parch','Fare', 'Ticket', 'Cabin', 'Embarked'], 1)

In [9]:
train_data.head()

,Survived,Pclass,Age,Person,Alone
0,0,3,22.0,1,1
1,1,1,38.0,2,1
2,1,3,26.0,2,0
3,1,1,35.0,2,1
4,0,3,35.0,1,0


In [10]:
t = train_data.iloc[:,0]
x = train_data.iloc[:,1:]

In [16]:
x = x.values

In [17]:
x = x.astype("f")
t = t.astype("i")

In [19]:
dataset = list(zip(x, t))

In [22]:
import chainer
import chainer.functions as F
import chainer.links as L
import chainer
import cupy

from chainer import training
from chainer.training import extensions

In [23]:
n_train = int(len(dataset)*0.7)

In [24]:
train, test = chainer.datasets.split_dataset_random(dataset, n_train, seed=1)

In [26]:
class NN(chainer.Chain):
    def __init__(self, n_mid_units1=5, n_out=2):
        super().__init__()
        with self.init_scope():
            self.fc1 = L.Linear(None, n_mid_units1)
            self.fc2 = L.Linear(None, n_out)
            
            self.bn = L.BatchNormalization(4)
    
    def __call__(self, x):
        h = self.bn(x)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.fc2(h)
        return h